#### Lecture 6-1 连接

在实际的数据处理中，通常需要把多个表中的数据按照某种连接方式进行整合。例如学生期末考试各个科目的成绩表按照$\color{red}{姓名}$和$\color{red}{班级}$连接成总的成绩表，又例如对企业员工的各类信息表按照$\color{red}{员工ID号}$进行连接汇总

主要包括关系连接和方向连接

In [1]:
import pandas as pd
import numpy as np

##### 一、 关系连接：

1. 关系连接：关系连接是指把两个表按照一组列或多组列的元素匹配结果进行连接，这些列被称为$\color{red}{键}$，往往用`on`参数表示。共有4种连接形式，分为左连接`left`、右连接`right`、内连接`inner`、外连接`outer`，如下图所示：

<img src="img/ch6_1.png" width="50%">

* 左连接：相当于以左侧的表中的key为依据进行整合，不在左侧表中key里的数据被剔除
* 右连接：相当于以右侧表中的key为依据进行整合，不在右侧表中key里的数据被提出。
* 内连接：相当于求两个表中公共的key
* 外连接：相当于求两个表中key的并集。

在panda上中，用参数how来表示。



上面这个简单的例子中，同一个表中的键没有出现重复的情况，那么如果出现重复的键应该如何处理？

只需把握一个原则，即只要两边同时出现的值，就以笛卡尔积的方式加入，如果单边出现则根据连接形式进行处理。其中，关于笛卡尔积可用如下例子说明：设左表中键`张三`出现两次，右表中的`张三`也出现两次，那么逐个进行匹配，最后产生的表必然包含`2*2`个姓名为`张三`的行。下面是一个对应例子的示意图：

<img src="img/ch6_2.png" width="60%">

##### 1. merge 值连接

在上面示意图中的例子中，两张表根据某一列的值来连接，事实上还可以通过几列值的组合进行连接，这种基于值的连接在`pandas`中可以由`merge`函数实现，

a) 简单的值连接：例如第一张图的左连接：

In [2]:
df1 = pd.DataFrame({'Name':['张三','李四'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['李四','王五'], 'Gender':['F','M']})
df1

,Name,Age
0,张三,20
1,李四,30


In [7]:
df2

,Name,Gender
0,李四,F
1,王五,M


In [3]:
df1.merge(df2, on='Name', how='left')

,Name,Age,Gender
0,张三,20,NaN
1,李四,30,F


b) 值相同，列名不同：如果两个表中想要连接的列不具备相同的列名，可以通过`left_on`和`right_on`指定，例如：

In [10]:
df1 = pd.DataFrame({'df1_name':['张三','李四'], 'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['李四','王五'], 'Gender':['F','M']})
df1.merge(df2, left_on='df1_name', right_on='df2_name', how='left')

,df1_name,Age,df2_name,Gender
0,张三,20,NaN,NaN
1,李四,30,李四,F


c) 重复列名：如果两个表中的列出现了重复的列名，那么可以通过`suffixes`参数指定。例如合并考试成绩的时候，第一个表记录了语文成绩，第二个是数学成绩：

In [11]:
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


d)多组键实现连接：在某些时候出现重复元素是麻烦的，例如两位同学来自不同的班级，但是姓名相同，这种时候就要指定`on`参数为多个列使得正确连接：

In [6]:
df1 = pd.DataFrame({'Name':['San Zhang', 'lisi'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'lisi'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'one']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,lisi,21,two


In [7]:
df2

,Name,Gender,Class
0,San Zhang,F,two
1,lisi,M,one


In [8]:
df1.merge(df2, on='Name', how='left') # 错误的结果

,Name,Age,Class_x,Gender,Class_y
0,San Zhang,20,one,F,two
1,lisi,21,two,M,one


In [9]:
df1.merge(df2, on=['Name', 'Class'], how='left') # 正确的结果

,Name,Age,Class,Gender
0,San Zhang,20,one,NaN
1,lisi,21,two,NaN


##### 二. 方向连接 concat()函数

值连接依赖`键`将两个表进行连接。在数据分析中，有时我们只想把两个或多个索引相同的表进行横向或者纵向拼接，此时可以使用concat()函数实现。

在`concat`中，最常用的有三个参数，
* `axis`:表示拼接方向
* `join`:连接形式
* `keys`:在新表中指示来自于哪一张旧表的名字。

a) 在默认状态下的`axis=0`，表示纵向拼接多个表，常常用于多个样本的拼接；而`axis=1`表示横向拼接多个表，常用于多个字段或特征的拼接。


In [23]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})
pd.concat([df1, df2]) #样本的拼接，具有相同的列

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


In [24]:
# 可以横向合并各表中的字段

df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
pd.concat([df1, df2, df3], 1)

/var/folders/l_/5lmrjkpx3xv28w5ky6d9w2mc0000gn/T/ipykernel_40392/1493175587.py:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat([df1, df2, df3], 1)


,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


b) join

虽然说`concat`是处理关系型合并的函数，但是它仍然是关于索引进行连接的。

join与merge()中的参数how含义一致，由于在多表中不存在左表和右表的概念，故参数join仅支持外连接和内连接。纵向拼接会根据列索引对其，默认状态下`join=outer`，表示保留所有的列，并将不存在的值设为缺失；`join=inner`，表示保留两个表都出现过的列。横向拼接则根据行索引对齐。

例如，对于外连接，在横向合并时取行索引元素的并集，在纵向合并时取列索引元素的并集。


In [22]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Gender':['M']})
pd.concat([df1, df2]) #默认join=outer,保留所有的列，不存在的列值设为缺失。

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,NaN
0,Wu Wang,NaN,M


In [13]:
df2 = pd.DataFrame({'Grade':[80, 90]}, index=[1, 2]) #索引号从1开始编号
df2

,Grade
1,80
2,90


In [14]:
pd.concat([df1, df2], 1)

/var/folders/l_/5lmrjkpx3xv28w5ky6d9w2mc0000gn/T/ipykernel_40392/2722207787.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat([df1, df2], 1)


,Name,Age,Grade
0,San Zhang,20.0,NaN
1,Si Li,30.0,80.0
2,NaN,NaN,90.0


In [16]:
df1

,Name,Age
0,San Zhang,20
1,Si Li,30


In [17]:
df2

,Grade
1,80
2,90


In [22]:
pd.concat([df1, df2], axis=1, join='inner')

,Name,Age,Grade
1,Si Li,30,80


因此，当确认要使用多表直接的方向合并时，尤其是横向的合并，可以先用`reset_index`方法恢复默认整数索引再进行合并，防止出现由索引的误对齐和重复索引的笛卡尔积带来的错误结果。

最后，`keys`参数的使用场景在于多个表合并后，用户仍然想要知道新表中的数据来自于哪个原表，这时可以通过`keys`参数产生多级索引进行标记。例如，第一个表中都是一班的同学，而第二个表中都是二班的同学，可以使用如下方式合并：

In [28]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
pd.concat([df1, df2], keys=['one', 'two'])

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

##### 三、课后作业
### Ex1：美国疫情数据集

现有美国4月12日至11月16日的疫情报表（在`/data/us_report`文件夹下），请将`New York`的`Confirmed, Deaths, Recovered, Active`合并为一张表，索引为按如下方法生成的日期字符串序列：

In [29]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()
date[:5]

['04-12-2020', '04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020']

通过学习通提交可执行的源代码。